In [ ]:
!pip install -U -q google-generativeai


In [ ]:
! pip3 install telebot
! pip install -q -U google-generativeai
! pip install Pillow
!pip3 install telebot

In [1]:
! pip3 install telebot
from telebot import TeleBot
import google.generativeai as genai
import PIL.Image
from io import BytesIO

bot = TeleBot("secret_key_telegram_API")
genai.configure(api_key="secret_key_gemini_key")
model = genai.GenerativeModel('gemini-pro-vision')

@bot.message_handler(commands=['start'])
def handle_first_message(message):
      bot.send_message(message.chat.id, "Welcome to the Meal Magic bot! send me a meal photo to tell you its ingredients & average calories..")


@bot.message_handler(content_types=['photo'])
def handle_image_message(message):
  loading_message = bot.send_message(message.chat.id, "Processing the image, please wait...")

  global img

  try:
    image_file = bot.get_file(message.photo[-1].file_id)
    image_data = bot.download_file(image_file.file_path)
    img = PIL.Image.open(BytesIO(image_data))

    response = model.generate_content(["Describe ingredients & average calories of a meal if it is a meal.", img], stream=True)
    response.resolve()
    image_description = response.text

    bot.send_message(message.chat.id, f"{image_description}")

    bot.send_message(message.chat.id, "Do you have any more questions about this? Ask me anything!")

  except Exception as e:
    bot.send_message(message.chat.id, f"Error processing the image: {str(e)}")

  finally:

    bot.delete_message(message.chat.id, loading_message.message_id)

@bot.message_handler(func=lambda message: True)
def handle_text_message(message):
    global img
    loading_message = bot.send_message(message.chat.id, "Processing, please wait...")
    if img:
        response = model.generate_content([message.text, img], stream=True)
        response.resolve()
        answer = response.text


        bot.send_message(message.chat.id, f"{answer}")
    else:

        bot.send_message(message.chat.id, "I'm not sure how to respond to that. Please send me a photo for analysis.")


    bot.delete_message(message.chat.id, loading_message.message_id)
bot.polling()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 3.2 MB/s eta 0:00:00
